# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [16]:
import numpy as np
import pandas as pd
import pickle
import math
from nltk.tokenize import sent_tokenize
import datetime

from IPython.display import Markdown, display

In [2]:
import sys
sys.path.append('..')

In [25]:
# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# From this project
# from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie

## Configuration

In [6]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../../datasets/'


In [7]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

In [20]:
SAMPLE_FRACTION = 0.04

test_dic = {'train': {}, 'test':{}}

for i in ['train','test']:
    for j in ['positive','negative']:
         test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .reset_index() \
            .reset_index() \
            .drop(['reviewerName', 'helpful', 'asin', 'index',
                   'summary', 'unixReviewTime', 'reviewTime'], axis=1) \
            .rename(columns={'level_0': 'asin'})


In [21]:
test_dic['train']['positive'].head(3)

,asin,reviewerID,reviewText,overall
0,0,A32244V7CQUBD6,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0
1,1,A32244V7CQUBD6,"This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;. There is the continuing plot from an earlier episode of some group on Earth trying to track them down. That's the main reason for Emmanuelle and Hafron to jump to different parts of the world. Otherwise, this episode is mostly sex scenes.",4.0
2,2,A33KKMGGVLZ29T,"This is an intimate concert of Robert Mirabal. Although I thought that it was, as I said, masterful, the sound, at times sounded a little muffled.The storytelling of the songs gave an insight of native culture and of Mirabal's own family stories and history.The Dance and Ee You Oo are my picks for the best songs, but they are all a joy to watch. The Rare Tribal Mob and the Mirabal Singers/Dancers are great and provide a mesmerising stage performance.Very enjoyable",4.0


In [22]:
total = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        print(test_dic[i][j].shape)
        total += test_dic[i][j].shape[0]
total

(5760, 4)
(5760, 4)
(1440, 4)
(1440, 4)


14400

## Remove objective sentences for case B

In [11]:
%reload_ext autoreload
obj_path = '../obj_subj_dev/'
fit_obj_tf = obj_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl'
fit_obj_model = obj_path + 'GBC_300_0.5_5_0.88cv.pkl'
subj_filter = SubjectiveFilter(fit_obj_tf, fit_obj_model)

In [12]:
# for tt in ['train','test']:
#     for pn in ['positive','negative']:
#         print(tt,pn,test_dic[tt][np].shape)
total = 0
for tt in test_dic.values():
    for df in tt.values():
#         display(df.head(1))
        total += df.shape[0]
        display(df.shape)
print(total)

(5760, 4)

(5760, 4)

(1440, 4)

(1440, 4)

14400


In [35]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

In [ ]:
CHUNK_SZ = 1

for REMOVE in ['subj', 'obj']:
    for REMOVE_FRACTION in [0.8, 0.6, 0.4, 0.2]:
        print('Starting computations for {} {}'.format(REMOVE, REMOVE_FRACTION))
        print (str(datetime.datetime.now()))
        sent_dfs = {'train':{},'test':{}}
        nb_sentences_removed = 0

        for ttname, tt in test_dic.items():
            for pn, df in tt.items():
                df_list = []
                start = 0
                while start < df.shape[0]:
                    end = start + CHUNK_SZ
                    df1 = df.iloc[start:end,:]
                    df2 = subj_filter.to_one_sent_per_row(df1)
                    df3, removed = subj_filter.transform(
                            df2,
                            'sentence', 
                            remove_fraction = REMOVE_FRACTION,
                            debug_level=0,
                            remove=REMOVE)
                    if removed == -1:
                        start = end
                        continue
                    df_list.append(df3)
                    nb_sentences_removed += removed
                    start = end

                if len(df_list) == 0:
                    sent_dfs[ttname][pn] = None
                    print('No reviews for {} {}'.format(ttname, pn))
                    continue
                sent_dfs[ttname][pn] = df_list.pop()
                while len(df_list) > 0:
                    sent_dfs[ttname][pn] = pd.merge(df_list.pop(), 
                                                    sent_dfs[ttname][pn], how='outer')
        # Save B
        print('Saving B for {} {}'.format(REMOVE, REMOVE_FRACTION))
        print (str(datetime.datetime.now()))
        pickle_out = open(data_path
                            + 'reviews_wout_top_' + str(int(round(REMOVE_FRACTION * 100))) + 'pct_' + REMOVE + '_B.pkl'
                            , "wb")
        pickle.dump(sent_dfs, pickle_out)
        pickle_out.close()

        # Create A with same number of reviews
        A_dic = {'train': {}, 'test':{}}

        for tt in ['train', 'test']:
            for pn in ['positive', 'negative']:
                df = test_dic[tt][pn]
                A_dic[tt][pn] = df[df['asin'].isin(sent_dfs[tt][pn]['asin'])]

        # Save A
        print('Saving A for {} {}'.format(REMOVE, REMOVE_FRACTION))
        print (str(datetime.datetime.now()))
        pickle_out = open(data_path
                        + 'reviews_wout_top_' + str(int(round(REMOVE_FRACTION * 100))) + 'pct_' + REMOVE + '_A.pkl'
                            , "wb")
        pickle.dump(A_dic, pickle_out)
        pickle_out.close()

Starting computations for subj 0.8
2019-07-23 22:07:38.683411
Saving B for subj 0.8
2019-07-23 22:30:59.980219
Saving A for subj 0.8
2019-07-23 22:31:00.023536
Starting computations for subj 0.6
2019-07-23 22:31:00.065450
